# Load libraries

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
from __future__ import absolute_import, division, print_function

# only keep warnings and errors
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='1'

import numpy as np
import argparse
import re
import time
import tensorflow as tf
import tensorflow.contrib.slim as slim

from monodepth_model import *
from monodepth_dataloader import *
from average_gradients import *

# Create dataset

In [ ]:
import glob

In [ ]:
left_files = sorted(glob.glob('/root/data/sflab_ground_truth/v2_071218/raw_images/left*'))[:250]
right_files = sorted(glob.glob('/root/data/sflab_ground_truth/v2_071218/raw_images/right*'))[:250]

In [ ]:
with open('/root/data/sflab_ground_truth/v2_071218/depth_map_files.txt', 'w') as f:
    for (left, right) in zip(left_files, right_files):
        f.write(left + ' ' + right + '\n')

# Display images

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

In [ ]:
ts = [int(os.path.basename(left).split('.')[0].split('_')[-1]) for left in left_files]

In [ ]:
plt.plot(ts)

In [ ]:
len(left_files)

In [ ]:
for _ in range(10): 
    ind = np.random.randint(low=0, high=len(left_files))
    print(ind)
    f, ax = plt.subplots(1, 2, figsize=(20, 10))
    ax[0].imshow(np.array(Image.open(left_files[ind]).resize((400, 300))))
    ax[1].imshow(np.array(Image.open(right_files[ind]).resize((400, 300))))
    plt.show()

### Useful functions

In [ ]:
def post_process_disparity(disp):
    if len(disp) == 1:
        _, h, w = disp.shape
        l_disp = disp[0,:,:]
        r_disp = np.fliplr(disp[1,:,:])
        m_disp = 0.5 * (l_disp + r_disp)
        l, _ = np.meshgrid(np.linspace(0, 1, w), np.linspace(0, 1, h))
        l_mask = 1.0 - np.clip(20 * (l - 0.05), 0, 1)
        r_mask = np.fliplr(l_mask)
        return r_mask * l_disp + l_mask * r_disp + (1.0 - l_mask - r_mask) * m_disp
    else:
        h, w = disp[0].squeeze().shape
        l_disp = disp[0].squeeze().astype(np.float32)
        r_disp = disp[1].squeeze().astype(np.float32)
        m_disp = 0.5 * (l_disp + r_disp)
        l, _ = np.meshgrid(np.linspace(0, 1, w), np.linspace(0, 1, h))
        l_mask = 1.0 - np.clip(20 * (l - 0.05), 0, 1)
        r_mask = np.fliplr(l_mask)
        return r_mask * l_disp + l_mask * r_disp + (1.0 - l_mask - r_mask) * m_disp

def count_text_lines(file_path):
    f = open(file_path, 'r')
    lines = f.readlines()
    f.close()
    return len(lines)


### Model parameters

In [ ]:
from collections import namedtuple

In [ ]:
parameters = namedtuple('parameters', 
                        'encoder, '
                        'height, width, '
                        'batch_size, '
                        'num_threads, '
                        'num_epochs, '
                        'do_stereo, '
                        'wrap_mode, '
                        'use_deconv, '
                        'alpha_image_loss, '
                        'disp_gradient_loss_weight, '
                        'lr_loss_weight, '
                        'full_summary, '
                       'filenames_file, '
                        'learning_rate, '
                         'data_path,'
                         'dataset,'
                         'mode, '
                         'num_gpus, '
                         'log_directory, '
                         'model_name, ')

In [ ]:
params = parameters(encoder='vgg',
                  height=256,
                  width=512,
                  batch_size=16,
                  num_threads=8,
                  num_epochs=500,
                  do_stereo=True,
                  wrap_mode='border',
                  use_deconv='True',
                  alpha_image_loss=0.85,
                  disp_gradient_loss_weight=0.1,
                  lr_loss_weight=1.0,
                  full_summary=True,
                  filenames_file='/root/data/sflab_ground_truth/v2_071218/depth_map_files.txt',
                  learning_rate=1e-4,
                  data_path='',
                  dataset='sflab',
                  mode='train',
                  num_gpus=2,
                  log_directory='/root/data/models/sflab/depthmap/',
                  model_name='sflab_0')

# Train loop

In [ ]:
import scipy.misc
import matplotlib.pyplot as plt
import copy

In [ ]:
f = open(params.filenames_file, 'r')
left_image_path, right_image_path = f.readline().split()
print('Left image path: {}'.format(left_image_path))
print('Right image path: {}'.format(right_image_path))

In [ ]:
"""Training loop."""
# LOAD IMAGES
left_input = scipy.misc.imread(left_image_path, mode="RGB")
original_height, original_width, num_channels = left_input.shape
left_input = scipy.misc.imresize(left_input, [params.height, params.width], interp='lanczos')
left_input_disp = copy.copy(left_input)
print(left_input.shape)
left_input = left_input.astype(np.float32) / 255
left_input = np.expand_dims(left_input, 0)

# input_images = np.stack((input_image, np.fliplr(input_image)), 0)
right_input = scipy.misc.imread(right_image_path, mode="RGB")
original_height, original_width, num_channels = right_input.shape
right_input = scipy.misc.imresize(right_input, [params.height, params.width], interp='lanczos')
right_input_disp = copy.copy(right_input)
right_input = right_input.astype(np.float32) / 255
right_input = np.expand_dims(right_input, 0)

# PLOTS
f, ax = plt.subplots(1, 2, figsize=(20,10))
ax[0].imshow(left_input_disp)
ax[1].imshow(right_input_disp)
plt.show()

with tf.Graph().as_default(), tf.device('/cpu:0'):

    global_step = tf.Variable(0, trainable=False)

    # OPTIMIZER
    num_training_samples = count_text_lines(params.filenames_file)

    steps_per_epoch = np.ceil(num_training_samples / params.batch_size).astype(np.int32)
    num_total_steps = params.num_epochs * steps_per_epoch
    print(steps_per_epoch)
    print(num_total_steps)
    start_learning_rate = params.learning_rate

    boundaries = [np.int32((3/5) * num_total_steps), np.int32((4/5) * num_total_steps)]
    values = [params.learning_rate, params.learning_rate / 2, params.learning_rate / 4]
    learning_rate = tf.train.piecewise_constant(global_step, boundaries, values)

    opt_step = tf.train.AdamOptimizer(learning_rate)

    print("total number of samples: {}".format(num_training_samples))
    print("total number of steps: {}".format(num_total_steps))

    dataloader = MonodepthDataloader(params.data_path, params.filenames_file, params, params.dataset, 
                                     params.mode)
    left  = dataloader.left_image_batch
    right = dataloader.right_image_batch
#         print(left)
#         print(right)
    # split for each gpu
    left_splits  = tf.split(left,  params.num_gpus, 0)
    right_splits = tf.split(right, params.num_gpus, 0)

    tower_grads  = []
    tower_losses = []
    reuse_variables = None
    
    print('loading model.....')
    with tf.variable_scope(tf.get_variable_scope()):
        for i in range(params.num_gpus):
            with tf.device('/gpu:%d' % i):

                model = MonodepthModel(params, params.mode, left_splits[i], right_splits[i], reuse_variables, i)

                loss = model.total_loss
                tower_losses.append(loss)

                reuse_variables = True

                grads = opt_step.compute_gradients(loss)

                tower_grads.append(grads)

    grads = average_gradients(tower_grads)

    apply_gradient_op = opt_step.apply_gradients(grads, global_step=global_step)

    total_loss = tf.reduce_mean(tower_losses)

    tf.summary.scalar('learning_rate', learning_rate, ['model_0'])
    tf.summary.scalar('total_loss', total_loss, ['model_0'])
    summary_op = tf.summary.merge_all('model_0')

    # SESSION
    config = tf.ConfigProto(allow_soft_placement=True)
    sess = tf.Session(config=config)

    # SAVER
    summary_writer = tf.summary.FileWriter(params.log_directory + '/' + params.model_name, sess.graph)
    train_saver = tf.train.Saver()

    # COUNT PARAMS
    total_num_parameters = 0
    for variable in tf.trainable_variables():
        total_num_parameters += np.array(variable.get_shape().as_list()).prod()
    print("number of trainable parameters: {}".format(total_num_parameters))

    # INIT
    sess.run(tf.global_variables_initializer())
    print('global variables initialized')
    sess.run(tf.local_variables_initializer())
    print('local variables initialized')
    coordinator = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coordinator)
    print('threads created')

#         # LOAD CHECKPOINT IF SET
#         if args.checkpoint_path != '':
#             train_saver.restore(sess, args.checkpoint_path.split(".")[0])

#             if args.retrain:
#                 sess.run(global_step.assign(0))

    # GO!
    print('go!')
    start_step = global_step.eval(session=sess)
    start_time = time.time()
    for step in range(start_step, num_total_steps):
        # print(step)
        before_op_time = time.time()
        _, loss_value = sess.run([apply_gradient_op, total_loss])
        duration = time.time() - before_op_time
        if step and step % 50 == 0:
            # some plot
#             disp = sess.run([model.disp_left_est[0], model.disp_right_est[0]], 
#             feed_dict={left: left_input, right: right_input})
#             disp_pp = post_process_disparity(disp)
#             plt.imshow(disp_pp, cmap='plasma')
#             plt.show()

            examples_per_sec = params.batch_size / duration
            time_sofar = (time.time() - start_time) / 3600
            training_time_left = (num_total_steps / step - 1.0) * time_sofar
            print_string = 'batch {:>6} | examples/s: {:4.2f} | loss: {:.5f} | time elapsed: {:.2f}h | time left: {:.2f}h'
            print(print_string.format(step, examples_per_sec, loss_value, time_sofar, training_time_left))
            summary_str = sess.run(summary_op)
            summary_writer.add_summary(summary_str, global_step=step)
        if step and step % 500 == 0:
            train_saver.save(sess, params.log_directory + '/' + params.model_name + '/model', global_step=step)

    train_saver.save(sess, params.log_directory + '/' + params.model_name + '/model', global_step=num_total_steps)


# Test loop

In [ ]:
from __future__ import absolute_import, division, print_function

# only keep warnings and errors
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='0'
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import numpy as np
import argparse
import re
import time
import tensorflow as tf
import tensorflow.contrib.slim as slim
import scipy.misc
import matplotlib.pyplot as plt
from PIL import Image

from monodepth_model import *
from monodepth_dataloader import *
from average_gradients import *

from collections import namedtuple

In [ ]:
def post_process_disparity(disp):
    if len(disp) == 1:
        _, h, w = disp.shape
        l_disp = disp[0,:,:]
        r_disp = np.fliplr(disp[1,:,:])
        m_disp = 0.5 * (l_disp + r_disp)
        l, _ = np.meshgrid(np.linspace(0, 1, w), np.linspace(0, 1, h))
        l_mask = 1.0 - np.clip(20 * (l - 0.05), 0, 1)
        r_mask = np.fliplr(l_mask)
        return r_mask * l_disp + l_mask * r_disp + (1.0 - l_mask - r_mask) * m_disp
    else:
        h, w = disp[0].squeeze().shape
        l_disp = disp[0].squeeze().astype(np.float32)
        r_disp = disp[1].squeeze().astype(np.float32)
        m_disp = 0.5 * (l_disp + r_disp)
        l, _ = np.meshgrid(np.linspace(0, 1, w), np.linspace(0, 1, h))
        l_mask = 1.0 - np.clip(20 * (l - 0.05), 0, 1)
        r_mask = np.fliplr(l_mask)
        return r_mask * l_disp + l_mask * r_disp + (1.0 - l_mask - r_mask) * m_disp

In [ ]:
parameters = namedtuple('parameters', 
                        'encoder, '
                        'height, width, '
                        'batch_size, '
                        'num_threads, '
                        'num_epochs, '
                        'do_stereo, '
                        'wrap_mode, '
                        'use_deconv, '
                        'alpha_image_loss, '
                        'disp_gradient_loss_weight, '
                        'lr_loss_weight, '
                        'full_summary, '
                       'filenames_file, '
                        'learning_rate, '
                         'data_path,'
                         'dataset,'
                         'mode, '
                         'num_gpus, '
                         'log_directory, '
                         'model_name, '
                         'checkpoint_path,')

In [ ]:
params = parameters(encoder='vgg',
                  height=256,
                  width=512,
                  batch_size=1,
                  num_threads=8,
                  num_epochs=10000,
                  do_stereo=True,
                  wrap_mode='border',
                  use_deconv='True',
                  alpha_image_loss=0.85,
                  disp_gradient_loss_weight=0.1,
                  lr_loss_weight=1.0,
                  full_summary=False,
                  filenames_file='/root/data/sflab_ground_truth/v2_071218/depth_map_files.txt',
                  learning_rate=1e-4,
                  data_path='',
                  dataset='kitti',
                  mode='test',
                  num_gpus=1,
                  log_directory='./logs',
                  model_name='overfit',
                  checkpoint_path='/root/data/models/sflab/depthmap/sflab_0/model-1000')

In [ ]:
f = open(params.filenames_file, 'r')

for (i, line) in enumerate(f):
    if i == 0:
        print("loading model")
        left  = tf.placeholder(tf.float32, [1, params.height, params.width, 3])
        right  = tf.placeholder(tf.float32, [1, params.height, params.width, 3])
        model = MonodepthModel(params, "test", left, right)

        # SESSION
        config = tf.ConfigProto(allow_soft_placement=True)
        sess = tf.Session(config=config)

        # SAVER
        train_saver = tf.train.Saver()

        # INIT
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        coordinator = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coordinator)

        # RESTORE
        print(params.checkpoint_path)
        restore_path = params.checkpoint_path # .split(".")[0]
        print(restore_path)
        train_saver.restore(sess, restore_path)
    
    left_image_path, right_image_path = line.split()
       
    # input_image = scipy.misc.imread(params.image_path, mode="RGB")
    # original_height, original_width, num_channels = input_image.shape
    # input_image = scipy.misc.imresize(input_image, [params.height, params.width], interp='lanczos')
    # input_image = input_image.astype(np.float32) / 255
    # input_images = np.stack((input_image, np.fliplr(input_image)), 0)
    left_input = scipy.misc.imread(left_image_path, mode="RGB")
    original_height, original_width, num_channels = left_input.shape
    left_input = scipy.misc.imresize(left_input, [params.height, params.width], interp='lanczos')
    left_input = left_input.astype(np.float32) / 255
    left_input = np.expand_dims(left_input, 0)
    # input_images = np.stack((input_image, np.fliplr(input_image)), 0)
    right_input = scipy.misc.imread(right_image_path, mode="RGB")
    original_height, original_width, num_channels = right_input.shape
    right_input = scipy.misc.imresize(right_input, [params.height, params.width], interp='lanczos')
    right_input = right_input.astype(np.float32) / 255
    right_input = np.expand_dims(right_input, 0)
    
    # disp = sess.run(model.disp_left_est[0], feed_dict={left: input_images})
    disp = sess.run([model.disp_left_est[0], model.disp_right_est[0]], feed_dict={left: left_input, right: right_input})
    print(len(disp))
    print('display')
    # disp_pp = post_process_disparity(disp.squeeze().astype(np.float32))
    disp_pp = post_process_disparity(disp)

    # output_directory = os.path.dirname(left_image_path)
    # output_name = os.path.splitext(os.path.basename(left_image_path))[0]

    # np.save(os.path.join(output_directory, "{}_disp.npy".format(output_name)), disp_pp)
    disp_to_img = scipy.misc.imresize(disp_pp.squeeze(), [original_height, original_width])
    plt.figure(figsize=(15, 15))
    plt.imshow(disp_to_img, cmap='plasma')
    plt.colorbar()
    plt.show()

In [ ]:
plt.imshow(disp_pp)

In [ ]:
disp